<a href="https://colab.research.google.com/github/Tom-atok/Tom-atok/blob/master/colab_notebooks/Speech2Text_kotoba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語音声書き起こし用Notebook

このノートブックは、日本語の音声を書き起こすために設計されており、Hugging Faceで提供されている[Kotoba Whisper](https://huggingface.co/kotoba-tech/kotoba-whisper-v1.0)を使用しています。シンプルでミニマムな構成になっており、音声データのアップロードから書き起こしまで、このノートブック内で全ての処理が完結するように設計されています。

## 事前準備

1. **Hugging Faceのトークン取得**: Kotoba Whisperを使用するために、Hugging Faceのアカウントを作成し、APIトークンを取得します。
2. **トークンの登録**: Google Colaboratoryのシークレット機能を使用して、取得したHugging Faceのトークンをノートブックで利用できるように設定します。
3. **ランタイム設定**: より高速な処理を行うため、Google ColaboratoryのランタイムをGPUに変更してください。上部メニューの「ランタイム」 > 「ランタイムのタイプを変更」から、ハードウェアアクセラレータとして「GPU」を選択します。

## 処理の流れ

1. **ライブラリのインストール**: 必要なライブラリ（Hugging Face関連ライブラリやKotoba Whisper用ライブラリ）をインストールします。
2. **ファイルのアップロード**: 書き起こし対象となる音声ファイルをアップロードします。
3. **モデルの構築**: Kotoba Whisperモデルをロードして準備します。
4. **モデルの実行**: アップロードした音声ファイルを使用して書き起こしを実行し、結果を出力します。


## 発展編
- google driveと接続して、特定のフォルダを全部文字起こしすることもできます。
  - [準備中]()

## ライブラリをインストールする

In [31]:
!pip install --upgrade pip
!pip install --upgrade transformers accelerate

In [32]:
import os
import torch
from google.colab import files
from pydub import AudioSegment
from pathlib import Path
from transformers import pipeline


## ファイルの準備

### ファイルをアップロードする

In [ ]:
# ファイルをアップロードする
uploaded = files.upload()

### ファイルのパスを取得し，データの形式を整える

In [34]:
# アップロードされたファイルの名前を取得
for filename in uploaded.keys():
    file_name_without_extension = os.path.splitext(filename)[0]
    # フルパスを作成（Colabのカレントディレクトリは'/content'です）
    file_path = os.path.join('/content', filename)
    print(f"Uploaded file is saved at: {file_path}")

# ファイルをの形式をwav形式に変更する
if not Path(file_path).suffix == ".wav":
    #ファイルの形式を任意のものからwav形式に変更する
    audio = AudioSegment.from_file(file_path)
    wav_path = f"/content/{file_name}.wav"
    audio.export(wav_path, format = "wav")

Saving sample_interview.WMA to sample_interview.WMA
Uploaded file is saved at: /content/sample_interview.WMA


In [38]:
# ファイルをの形式をwav形式に変更する
if not Path(file_path).suffix == ".wav":
    #ファイルの形式を任意のものからwav形式に変更する
    audio = AudioSegment.from_file(file_path)
    wav_path = f"/content/{file_name}.wav"
    audio.export(wav_path, format = "wav")

## 書き起こし

### モデル構築
Huggin_Faceのトークンを登録していないとここでエラーが出る

In [39]:
# config
model_id = "kotoba-tech/kotoba-whisper-v1.0"
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_kwargs = {"attn_implementation": "sdpa"} if torch.cuda.is_available() else {}
generate_kwargs = {"language": "ja", "task": "transcribe"}

# load model
pipe = pipeline(
    "automatic-speech-recognition",
    model=model_id,
    torch_dtype=torch_dtype,
    device=device,
    model_kwargs=model_kwargs
)

### モデルの実行

In [ ]:
result = pipe(wav_path,return_timestamps=True, generate_kwargs=generate_kwargs)
print(result["text"])

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


## データを保存する

In [ ]:
#outputフォルダにtxt形式で保存
path_output = f'/content/{file_name_without_extension}.txt'
with open(path_output, 'w') as f:
  f.write(result["text"])